In [13]:
import re

def clean_text(text):

    text = re.sub(r'\*\*page \d+\*\*', '', text)
    text = re.sub(r'\*\*question \d+\*\*', '', text)
    text = re.sub(r'\*\*answer \d+\*\*', '', text)
    # Step 1: 去除網址和 EMAIL
    text = re.sub(r'http\S+|www\S+|https\S+|[\w\.-]+@[\w\.-]+', '', text)

    text = re.sub(r'【[A-Za-z0-9]+】', '', text)

    # Step 6: 去除 "第 X 頁，共 Y 頁" 格式
    text = re.sub(r'第 \d+ 頁，共 \d+ 頁', '', text)

    # Step 7: 去除 "X/Y" 或 "X / Y" 格式
    text = re.sub(r'\b\d+ ?/ ?\d+\b', '', text)

    # Step 8: 去除 "~X~" 格式
    text = re.sub(r'~\d+~', '', text)

    # Step 9: 去除 "（接次頁）" 和 "（承前頁）"
    text = re.sub(r'（接次頁）|（承前頁）', '', text)

    # Step 10: 去除 "- X -" 格式
    text = re.sub(r'- \d+ -', '', text)

    # Step 2: 去除無意義數字（可以依需求調整，如果想保留某些數字格式）
    text = re.sub(r'\b\d+\b', '', text)

    # Step 3: 去除標點符號
    text = re.sub(r'[^\w\s]', '', text)

    # 去除多餘的空格
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 測試範例
sample_text = """
這是範例 https://example.com 測試 email@example.com 12345 測試！ **page 1 ** 其他內容 【APMOPL】
第 11 頁，共 24 頁
第 12 頁，共 24 頁
3/14
5/22
12 / 17
【APKQTL】-11/16
~7~
（接次頁）
（承前頁）
- 15 -
結尾。
"""
cleaned_text = clean_text(sample_text)
print(cleaned_text)


這是範例 測試 測試 page 其他內容 結尾


In [ ]:
from tqdm import tqdm_notebook
from pathlib import Path
from opencc import OpenCC
import re

count = 0
for folder in Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text').iterdir():
    if folder.is_dir():
        for file in folder.iterdir():
            count += 1
pbar = tqdm_notebook(total=count)
opencc = OpenCC("s2t")

output_path = Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text_cleaned')
output_path.mkdir(parents=True, exist_ok=True)

for folder in Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text').iterdir():
    if folder.is_dir():
        output_folder = output_path / folder.name
        output_folder.mkdir(parents=True, exist_ok=True)
        for file in folder.iterdir():
            output_file = output_folder / file.name
            with open(file, 'r') as f, open(output_file, 'w') as f_out:
                text = f.read()
                text = opencc.convert(text)
                texts = text.split('\n')
                for text in texts:
                    text = clean_text(text)
                    f_out.write(text + '\n')
            pbar.update(1)


In [14]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")

In [15]:
def get_ckip_ws(text):
    wss = ws_driver(text)
    result = []
    for ws in wss:
        result.append(' '.join(ws))

    return result

texts = [
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會",
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會",
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會",
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會",
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會",
]
res = get_ckip_ws(texts)
print(res)


Inference: 100%|██████████| 1/1 [00:00<00:00,  7.47it/s]

['美國 參議院 針對 今天 總統 布什 所 提名 的 勞工部長 趙小蘭 展開 認可 聽證會', '美國 參議院 針對 今天 總統 布什 所 提名 的 勞工部長 趙小蘭 展開 認可 聽證會', '美國 參議院 針對 今天 總統 布什 所 提名 的 勞工部長 趙小蘭 展開 認可 聽證會', '美國 參議院 針對 今天 總統 布什 所 提名 的 勞工部長 趙小蘭 展開 認可 聽證會', '美國 參議院 針對 今天 總統 布什 所 提名 的 勞工部長 趙小蘭 展開 認可 聽證會']


In [ ]:
from tqdm import tqdm_notebook
from pathlib import Path
from opencc import OpenCC

count = 0
for folder in Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text_cleaned').iterdir():
    if folder.is_dir():
        for file in folder.iterdir():
            count += 1
pbar = tqdm_notebook(total=count)
opencc = OpenCC("s2t")

output_path = Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text_cleaned_ckip_converted')
output_path.mkdir(parents=True, exist_ok=True)

for folder in Path('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/reference_text_cleaned').iterdir():
    if folder.is_dir() and folder.name == "finance":
        output_folder = output_path / folder.name
        output_folder.mkdir(parents=True, exist_ok=True)
        for file in folder.iterdir():
            output_file = output_folder / file.name
            with open(file, 'r') as f, open(output_file, 'w') as f_out:
                text = f.read()
                if text.strip() == "":
                    text = 'empty'
                converted_text = opencc.convert(text)
                converted_text = converted_text.replace(" ", "")
                converted_text = converted_text.split('\n')
                converted_text = get_ckip_ws(converted_text)
                converted_text = '\n'.join(converted_text)
                f_out.write(converted_text)
            pbar.update(1)


In [17]:
import json

with open('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/dataset/preliminary/questions_example.json', 'r') as f:
    data = json.load(f)
questions = data['questions']

for question in questions:
    question['query'] = clean_text(question['query'])
    question['query'] = get_ckip_ws([question['query']])[0]

with open('/Users/goodweather/Desktop/workspace/tbrain_2024/source/競賽資料集/dataset/preliminary/questions_example_cleaned_ckip.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)



Inference: 100%|██████████| 1/1 [00:00<00:00, 27.60it/s]
